In [1]:
import numpy as np
import pandas as pd
import pandas_profiling as pp

import warnings
warnings.filterwarnings('ignore')

from geopy.distance import geodesic

df = pd.read_csv("data_hackathon_v04.csv", sep="|", parse_dates=["created_at"], index_col="ride_id")

In [2]:
from h3 import h3

df["cell_id"] = df[["pickup_lat","pickup_lng"]].apply(lambda x: h3.geo_to_h3(*x, 7), axis=1)

In [3]:
def make_df_time(df):
    df_time = df[["created_at"]]

    df_time["day"] = df_time.created_at.dt.day
    df_time["month"] = df_time.created_at.dt.month
    df_time["year"] = df_time.created_at.dt.year
    df_time["is_year_start"] = np.int8(df_time.created_at.dt.is_year_start)
    df_time["is_year_end"] = np.int8(df_time.created_at.dt.is_year_end)
    df_time["dayofyear"] = df_time.created_at.dt.dayofyear
    df_time["dayofweek"] = df_time.created_at.dt.dayofweek

    df_time["week"] = df_time.created_at.dt.week

    df_time["hour"] = df_time.created_at.dt.hour
    df_time["minute"] = df_time.created_at.dt.minute
    df_time["final_time"] = (df_time["hour"] + df_time["minute"] // 30) % 24
    df_time["dayofweek"] = df_time.created_at.dt.dayofweek
    df_time["is_weekend"] = df_time["dayofweek"] // 5

    df_time.drop(["created_at"], inplace=True, axis=1)
    return df_time

def in_square(x, y):
    square = (49.895468, 23.907734, 49.769849, 24.121698)
    return (square[2] <= x <= square[0]) and (square[1] <= y <= square[3])

def clear_df(df):
    df["in_Lviv"] = (df[["pickup_lat", "pickup_lng"]].apply(lambda x: in_square(*x), axis=1) & df[["dropoff_lat", "dropoff_lng"]].apply(lambda x: in_square(*x), axis=1))
    df = df[df["in_Lviv"]]
    df["simple_distance"] = np.array(list(map(lambda x: geodesic(x[:2], x[2:]).km, 
                                          df[["pickup_lat", "pickup_lng", "dropoff_lat", "dropoff_lng"]].values)))
    df = df[abs(df["simple_distance"] - df["ride_distance"]) < 10]
    df.drop(["in_Lviv", "simple_distance"], axis=1, inplace=True)
    return df

def add_agg(df_in, df_out, group_by, name, aggs, add_name):
    if "mode" in aggs:
        aggs.remove("mode")
        _ = df_in.groupby(group_by)[name].agg(aggs)
        _["mode"] = df_in.groupby(group_by)[name].apply(lambda x: np.bincount(x).argmax())
    else:
        _ = df_in.groupby(group_by)[name].agg(aggs)
    _.columns = ["_".join([add_name, group_by, name, column]) for column in _.columns]
    return df_out.merge(_, left_index=True, right_index=True, how="left")

def make_aggs(df, name, days, LAST_DAY):
    df__ = pd.DataFrame(index=df[name].unique())

    df_ = df[(LAST_DAY - df.created_at).dt.days > 0]
    df__ = add_agg(df_, df__, name, "canceled_by_client", ["mean", "median", "std", "count"], "all")
    df__ = df__[df__[f"all_{name}_canceled_by_client_count"] > days]
    df__ = add_agg(df_, df__, name, "canceled_by_driver", ["mean", "median", "std"], "all")
    df__ = add_agg(df_, df__, name, "ride_to_suburb", ["mean", "std"], "all")
    df__ = add_agg(df_, df__, name, 'ride_distance', ["mean", "std", "median", "min", "max"], "all")
    df__ = add_agg(df_, df__, name, 'dayofweek', ["median", "mode"], "all")
    df__ = add_agg(df_, df__, name, 'final_time', ["median", "mode"], "all")
    df__ = add_agg(df_, df__, name, 'is_weekend', ["median", "std", "mean", "mode"], "all")
    
    df_ = df[(LAST_DAY - df.created_at).dt.days // 8 == 0]
    df__ = add_agg(df_, df__, name, "canceled_by_client", ["mean", "median", "std", "count"], "3_month")
    df__ = add_agg(df_, df__, name, "canceled_by_driver", ["mean", "median", "std"], "3_month")
    df__ = add_agg(df_, df__, name, "ride_to_suburb", ["mean", "std"], "3_month")
    df__ = add_agg(df_, df__, name, 'ride_distance', ["mean", "std", "median", "min", "max"], "3_month")
    df__ = add_agg(df_, df__, name, 'dayofweek', ["median", "mode"], "3_month")
    df__ = add_agg(df_, df__, name, 'final_time', ["median", "mode"], "3_month")
    df__ = add_agg(df_, df__, name, 'is_weekend', ["median", "std", "mean", "mode"], "3_month")

    df_ = df[(LAST_DAY - df.created_at).dt.days // 6 == 0]
    df__ = add_agg(df_, df__, name, "canceled_by_client", ["mean", "median", "std", "count"], "3_month")
    df__ = add_agg(df_, df__, name, "canceled_by_driver", ["mean", "median", "std"], "3_month")
    df__ = add_agg(df_, df__, name, "ride_to_suburb", ["mean", "std"], "3_month")
    df__ = add_agg(df_, df__, name, 'ride_distance', ["mean", "std", "median", "min", "max"], "3_month")
    df__ = add_agg(df_, df__, name, 'dayofweek', ["median", "mode"], "3_month")
    df__ = add_agg(df_, df__, name, 'final_time', ["median", "mode"], "3_month")
    df__ = add_agg(df_, df__, name, 'is_weekend', ["median", "std", "mean", "mode"], "3_month")

    df_ = df[(LAST_DAY - df.created_at).dt.days // 4 == 0]
    df__ = add_agg(df_, df__, name, "canceled_by_client", ["mean", "median", "std", "count"], "month")
    df__ = add_agg(df_, df__, name, "canceled_by_driver", ["mean", "median", "std"], "month")
    df__ = add_agg(df_, df__, name, "ride_to_suburb", ["mean", "std"], "month")
    df__ = add_agg(df_, df__, name, 'ride_distance', ["mean", "std", "median", "min", "max"], "month")
    df__ = add_agg(df_, df__, name, 'dayofweek', ["median", "mode"], "month")
    df__ = add_agg(df_, df__, name, 'final_time', ["median", "mode"], "month")
    df__ = add_agg(df_, df__, name, 'is_weekend', ["median", "std", "mean", "mode"], "month")
    df__[f"{name}_active"] = df__.index.isin(df_[name])

    df_ = df[(LAST_DAY - df.created_at).dt.days // 3 == 0]
    df__ = add_agg(df_, df__, name, "canceled_by_client", ["mean", "median", "std", "count"], "week")
    df__ = add_agg(df_, df__, name, "canceled_by_driver", ["mean", "median", "std"], "week")
    df__ = add_agg(df_, df__, name, "ride_to_suburb", ["mean", "std"], "week")
    df__ = add_agg(df_, df__, name, 'ride_distance', ["mean", "std", "median", "min", "max"], "week")
    df__ = add_agg(df_, df__, name, 'dayofweek', ["median", "mode"], "week")
    df__ = add_agg(df_, df__, name, 'final_time', ["median", "mode"], "week")
    df__ = add_agg(df_, df__, name, 'is_weekend', ["median", "std", "mean", "mode"], "week")

    df_ = df[(LAST_DAY - df.created_at).dt.days // 2 == 0]
    df__ = add_agg(df_, df__, name, "canceled_by_client", ["mean", "median", "std", "count"], "day")
    df__ = add_agg(df_, df__, name, "canceled_by_driver", ["mean", "median", "std"], "day")
    df__ = add_agg(df_, df__, name, "ride_to_suburb", ["mean", "std"], "day")
    df__ = add_agg(df_, df__, name, 'ride_distance', ["mean", "std", "median", "min", "max"], "day")
    df__ = add_agg(df_, df__, name, 'dayofweek', ["median", "mode"], "day")
    df__ = add_agg(df_, df__, name, 'final_time', ["median", "mode"], "day")
    df__ = add_agg(df_, df__, name, 'is_weekend', ["median", "std", "mean", "mode"], "day")
    print(df__.shape)
    df__.fillna(0.0, inplace=True)
    return df__

In [4]:
%%time
df = clear_df(df)

LAST_DAY = df.created_at.max()
LAST_DAY

CPU times: user 3min 23s, sys: 208 ms, total: 3min 24s
Wall time: 3min 24s


In [5]:
%%time
df_time = make_df_time(df)
df = df.merge(df_time, how="left", left_index=True, right_index=True)

CPU times: user 1.02 s, sys: 88 ms, total: 1.11 s
Wall time: 1.1 s


In [6]:
# %%time
# df_driver = make_aggs(df, "driver_id", 30)
# %%time
# df_user = make_aggs(df, "user_id", 10)
# df = df.merge(df_driver, how="left", left_on="driver_id", right_index=True)
# df = df.merge(df_user, how="left", left_on="user_id", right_index=True)

In [7]:
df.dropna(0, inplace=True)
df_cell = make_aggs(df, "cell_id", 26, LAST_DAY)
df = df[df.cell_id.isin(df_cell.index)]
df.to_csv("new_csv.csv")
df_cell

(43, 133)


,all_cell_id_canceled_by_client_mean,all_cell_id_canceled_by_client_median,all_cell_id_canceled_by_client_std,all_cell_id_canceled_by_client_count,all_cell_id_canceled_by_driver_mean,all_cell_id_canceled_by_driver_median,all_cell_id_canceled_by_driver_std,all_cell_id_ride_to_suburb_mean,all_cell_id_ride_to_suburb_std,all_cell_id_ride_distance_mean,...,day_cell_id_ride_distance_min,day_cell_id_ride_distance_max,day_cell_id_dayofweek_median,day_cell_id_dayofweek_mode,day_cell_id_final_time_median,day_cell_id_final_time_mode,day_cell_id_is_weekend_median,day_cell_id_is_weekend_std,day_cell_id_is_weekend_mean,day_cell_id_is_weekend_mode
871e7689cffffff,0.041075,0,0.198466,100717,0.002025,0,0.044960,0.001747,0.041766,4.648376,...,0.020000,15.270000,2.0,2.0,16.0,19.0,0.0,0.0,0.0,0.0
871e76882ffffff,0.033453,0,0.179818,67707,0.001935,0,0.043944,0.001167,0.034139,5.023944,...,0.220000,15.580000,2.0,2.0,15.0,19.0,0.0,0.0,0.0,0.0
871e7689dffffff,0.040810,0,0.197851,76452,0.002001,0,0.044691,0.002433,0.049265,4.970251,...,0.840000,14.780000,2.0,2.0,14.0,18.0,0.0,0.0,0.0,0.0
871e768b3ffffff,0.062723,0,0.242496,3651,0.001643,0,0.040511,0.002465,0.049595,9.403481,...,3.900000,14.700000,2.0,2.0,11.0,9.0,0.0,0.0,0.0,0.0
871e7689bffffff,0.032218,0,0.176581,26383,0.001516,0,0.038909,0.002464,0.049576,6.913683,...,0.990000,13.310000,2.0,2.0,12.0,9.0,0.0,0.0,0.0,0.0
871e76126ffffff,0.038239,0,0.191778,19012,0.001841,0,0.042868,0.003471,0.058819,7.534724,...,0.600000,14.510000,2.0,2.0,11.0,9.0,0.0,0.0,0.0,0.0
871e76883ffffff,0.049001,0,0.215871,236483,0.002947,0,0.054210,0.002186,0.046706,4.932625,...,0.020000,14.710000,2.0,2.0,16.0,23.0,0.0,0.0,0.0,0.0
871e76895ffffff,0.039155,0,0.193976,8428,0.000593,0,0.024351,0.001780,0.042152,6.678244,...,0.520000,13.110000,2.0,2.0,11.0,11.0,0.0,0.0,0.0,0.0
871e76898ffffff,0.034685,0,0.182984,35606,0.001292,0,0.035921,0.001657,0.040673,5.574838,...,1.480000,14.410000,2.0,2.0,14.0,19.0,0.0,0.0,0.0,0.0
871e76880ffffff,0.031094,0,0.173573,42195,0.001351,0,0.036730,0.001825,0.042680,5.485295,...,0.060000,15.430000,2.0,2.0,11.0,9.0,0.0,0.0,0.0,0.0


In [8]:
df_cell["all_cell_id_canceled_by_client_count"]

871e7689cffffff    100717
871e76882ffffff     67707
871e7689dffffff     76452
871e768b3ffffff      3651
871e7689bffffff     26383
871e76126ffffff     19012
871e76883ffffff    236483
871e76895ffffff      8428
871e76898ffffff     35606
871e76880ffffff     42195
871e76899ffffff     33689
871e76881ffffff     37953
871e7689affffff     33927
871e76124ffffff      2906
871e76134ffffff     12760
871e7689effffff     85399
871e76891ffffff     43746
871e76890ffffff      4678
871e76135ffffff      7406
871e76893ffffff     15215
871e7688affffff     16888
871e7688effffff     18577
871e76885ffffff      6301
871e76896ffffff       827
871e76c69ffffff       856
871e76892ffffff      2348
871e76c6dffffff      3400
871e768aaffffff      3310
871e7688cffffff       536
871e76888ffffff      1931
871e76886ffffff      1009
871e768abffffff       406
871e76122ffffff       351
871e76c68ffffff       309
871e76894ffffff       244
871e768b1ffffff       274
871e76c6cffffff       187
871e768a8ffffff        73
871e7688bfff

In [9]:
df.shape

(957478, 24)

In [10]:
center = list(df_cell.index)
center_size = len(center)

In [11]:
df_ = df[df["cell_id"].isin(center)]

In [12]:
from datetime import timedelta

from keras.utils import to_categorical

Using TensorFlow backend.


In [13]:
day = LAST_DAY.date()
day = day - timedelta(0)

In [22]:
day_nums = 30
X = []
y = np.zeros((24, day_nums, center_size))
for k in range(0, day_nums):
    day = LAST_DAY.date() - timedelta(k + 22)
    redundant_cols=["hour", "minute", "final_time"]

    df__time = make_df_time(pd.DataFrame({"created_at": [pd.Timestamp(day)]}))
    df__time.drop(redundant_cols, axis=1, inplace=True)
    df__time = np.int16(np.concatenate((to_categorical(df__time["month"], 12)[0],
                   to_categorical(df__time["dayofweek"], 7)[0],
                   df__time.drop(["month", "dayofweek"], axis=1).values[0])))
    
    
    res = make_aggs(df_, "cell_id", 0, day).values.reshape(1, -1)[0]
    X.append(np.concatenate((df__time, res)).reshape(1, -1))
    
    group = df_[df_.created_at.dt.date == day].groupby(["final_time", "cell_id"]).created_at.count()
    for i in range(24):
        for j, cell in enumerate(center):
            if i in group:
                if cell in group.loc[i]:
                    y[i][k][j] = group.loc[i, cell]

(43, 133)
(43, 133)
(43, 133)
(43, 133)
(43, 133)
(43, 133)
(43, 133)
(43, 133)
(43, 133)
(43, 133)
(43, 133)
(43, 133)
(43, 133)
(43, 133)
(43, 133)
(43, 133)
(43, 133)
(43, 133)
(43, 133)
(43, 133)
(43, 133)
(43, 133)
(43, 133)
(43, 133)
(43, 133)
(43, 133)
(43, 133)
(43, 133)
(43, 133)
(43, 133)


In [23]:
X = np.array(X)
X.shape

(30, 1, 5745)

In [24]:
y.shape

(24, 30, 43)

In [25]:
from keras.layers import Dense, Flatten, LSTM, Input, BatchNormalization, Dropout
from keras.models import Model
import keras.backend as K 

In [46]:
inp = Input(tuple(X.shape[1:]))
lstm1 = LSTM(units=128, activation="relu", return_sequences=True)(inp)
# lstm2 = LSTM(units=128, activation="relu", return_sequences=True)(lstm1)
outs= []
for i in range(y.shape[0]):
    flt = Flatten()(lstm1)
    dns = Dense(units=128, activation="relu")(flt)
    bn = BatchNormalization()(dns)
    outs.append(Dense(center_size, activation="relu")(bn))
model = Model(inp, outs)

In [47]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 1, 5745)      0                                            
__________________________________________________________________________________________________
lstm_7 (LSTM)                   (None, 1, 128)       3007488     input_5[0][0]                    
__________________________________________________________________________________________________
flatten_97 (Flatten)            (None, 128)          0           lstm_7[0][0]                     
__________________________________________________________________________________________________
flatten_98 (Flatten)            (None, 128)          0           lstm_7[0][0]                     
__________________________________________________________________________________________________
flatten_99

In [48]:
from keras.metrics import mean_squared_logarithmic_error

In [ ]:
model = Model(inp, outs)
def rmsle(y_true, y_pred):
        return K.sqrt(K.mean(K.square(K.log(y_pred + 1.) - K.log(y_true + 1.)), axis=-1)) 

model.compile(optimizer = "adam", loss = mean_squared_logarithmic_error, metrics =["accuracy"])

In [ ]:
model.fit(X, list(y), batch_size=2, epochs=10000)

Epoch 1/10000
30/30 [==============================] - 19s 626ms/step - loss: 49.7594 - dense_194_loss: 2.2443 - dense_196_loss: 1.7175 - dense_198_loss: 1.3085 - dense_200_loss: 0.8866 - dense_202_loss: 0.7293 - dense_204_loss: 0.9202 - dense_206_loss: 0.8346 - dense_208_loss: 0.9751 - dense_210_loss: 2.1409 - dense_212_loss: 2.8631 - dense_214_loss: 2.9755 - dense_216_loss: 2.6277 - dense_218_loss: 2.1128 - dense_220_loss: 2.0660 - dense_222_loss: 2.1180 - dense_224_loss: 2.1448 - dense_226_loss: 2.3068 - dense_228_loss: 2.5740 - dense_230_loss: 2.9851 - dense_232_loss: 2.8780 - dense_234_loss: 2.6863 - dense_236_loss: 2.4457 - dense_238_loss: 2.6098 - dense_240_loss: 2.6089 - dense_194_acc: 0.0000e+00 - dense_196_acc: 0.0000e+00 - dense_198_acc: 0.0000e+00 - dense_200_acc: 0.4000 - dense_202_acc: 0.0000e+00 - dense_204_acc: 0.0000e+00 - dense_206_acc: 0.1333 - dense_208_acc: 0.0000e+00 - dense_210_acc: 0.1667 - dense_212_acc: 0.0000e+00 - dense_214_acc: 0.0000e+00 - dense_216_acc: 0

30/30 [==============================] - 1s 49ms/step - loss: 37.0116 - dense_194_loss: 1.6787 - dense_196_loss: 1.2638 - dense_198_loss: 0.9963 - dense_200_loss: 0.6931 - dense_202_loss: 0.6046 - dense_204_loss: 0.6912 - dense_206_loss: 0.6388 - dense_208_loss: 0.6901 - dense_210_loss: 1.5661 - dense_212_loss: 2.0908 - dense_214_loss: 2.2125 - dense_216_loss: 1.9442 - dense_218_loss: 1.5251 - dense_220_loss: 1.5918 - dense_222_loss: 1.5396 - dense_224_loss: 1.5698 - dense_226_loss: 1.6901 - dense_228_loss: 1.8777 - dense_230_loss: 2.2242 - dense_232_loss: 2.1915 - dense_234_loss: 2.0186 - dense_236_loss: 1.8506 - dense_238_loss: 1.9283 - dense_240_loss: 1.9341 - dense_194_acc: 0.0333 - dense_196_acc: 0.1000 - dense_198_acc: 0.5000 - dense_200_acc: 0.5000 - dense_202_acc: 0.4667 - dense_204_acc: 0.4000 - dense_206_acc: 0.3667 - dense_208_acc: 0.0667 - dense_210_acc: 0.2667 - dense_212_acc: 0.1333 - dense_214_acc: 0.1667 - dense_216_acc: 0.0333 - dense_218_acc: 0.4667 - dense_220_acc: 0

30/30 [==============================] - 1s 41ms/step - loss: 34.1875 - dense_194_loss: 1.5093 - dense_196_loss: 1.1801 - dense_198_loss: 0.9118 - dense_200_loss: 0.6578 - dense_202_loss: 0.5956 - dense_204_loss: 0.5525 - dense_206_loss: 0.5840 - dense_208_loss: 0.6458 - dense_210_loss: 1.4066 - dense_212_loss: 1.9299 - dense_214_loss: 2.0539 - dense_216_loss: 1.7701 - dense_218_loss: 1.4099 - dense_220_loss: 1.4398 - dense_222_loss: 1.4452 - dense_224_loss: 1.4870 - dense_226_loss: 1.5822 - dense_228_loss: 1.7574 - dense_230_loss: 2.0705 - dense_232_loss: 2.0251 - dense_234_loss: 1.8592 - dense_236_loss: 1.7293 - dense_238_loss: 1.7966 - dense_240_loss: 1.7878 - dense_194_acc: 0.4000 - dense_196_acc: 0.0333 - dense_198_acc: 0.5000 - dense_200_acc: 0.4667 - dense_202_acc: 0.4667 - dense_204_acc: 0.3667 - dense_206_acc: 0.5000 - dense_208_acc: 0.2000 - dense_210_acc: 0.1667 - dense_212_acc: 0.1667 - dense_214_acc: 0.1000 - dense_216_acc: 0.1667 - dense_218_acc: 0.5000 - dense_220_acc: 0

30/30 [==============================] - 1s 40ms/step - loss: 32.8764 - dense_194_loss: 1.4193 - dense_196_loss: 1.1563 - dense_198_loss: 0.8846 - dense_200_loss: 0.6592 - dense_202_loss: 0.5738 - dense_204_loss: 0.2802 - dense_206_loss: 0.5604 - dense_208_loss: 0.6440 - dense_210_loss: 1.3568 - dense_212_loss: 1.8878 - dense_214_loss: 1.9531 - dense_216_loss: 1.6936 - dense_218_loss: 1.3947 - dense_220_loss: 1.4182 - dense_222_loss: 1.3919 - dense_224_loss: 1.4381 - dense_226_loss: 1.5320 - dense_228_loss: 1.6891 - dense_230_loss: 2.0102 - dense_232_loss: 1.9700 - dense_234_loss: 1.7987 - dense_236_loss: 1.6646 - dense_238_loss: 1.7663 - dense_240_loss: 1.7337 - dense_194_acc: 0.5000 - dense_196_acc: 0.4333 - dense_198_acc: 0.5000 - dense_200_acc: 0.4667 - dense_202_acc: 0.4667 - dense_204_acc: 0.5333 - dense_206_acc: 0.6333 - dense_208_acc: 0.3333 - dense_210_acc: 0.2333 - dense_212_acc: 0.3333 - dense_214_acc: 0.1333 - dense_216_acc: 0.3333 - dense_218_acc: 0.5000 - dense_220_acc: 0

30/30 [==============================] - 1s 45ms/step - loss: 32.1296 - dense_194_loss: 1.3642 - dense_196_loss: 1.1228 - dense_198_loss: 0.8230 - dense_200_loss: 0.6342 - dense_202_loss: 0.5521 - dense_204_loss: 0.2528 - dense_206_loss: 0.5529 - dense_208_loss: 0.6314 - dense_210_loss: 1.2889 - dense_212_loss: 1.8660 - dense_214_loss: 1.9092 - dense_216_loss: 1.6916 - dense_218_loss: 1.3850 - dense_220_loss: 1.4153 - dense_222_loss: 1.3391 - dense_224_loss: 1.4500 - dense_226_loss: 1.4913 - dense_228_loss: 1.6629 - dense_230_loss: 1.9597 - dense_232_loss: 1.9132 - dense_234_loss: 1.7630 - dense_236_loss: 1.6401 - dense_238_loss: 1.7340 - dense_240_loss: 1.6868 - dense_194_acc: 0.5000 - dense_196_acc: 0.5000 - dense_198_acc: 0.5000 - dense_200_acc: 0.4667 - dense_202_acc: 0.4667 - dense_204_acc: 0.8333 - dense_206_acc: 0.6333 - dense_208_acc: 0.3667 - dense_210_acc: 0.2000 - dense_212_acc: 0.3000 - dense_214_acc: 0.2000 - dense_216_acc: 0.3667 - dense_218_acc: 0.4667 - dense_220_acc: 0

30/30 [==============================] - 1s 37ms/step - loss: 31.4911 - dense_194_loss: 1.0486 - dense_196_loss: 1.1252 - dense_198_loss: 0.8137 - dense_200_loss: 0.6390 - dense_202_loss: 0.5500 - dense_204_loss: 0.2344 - dense_206_loss: 0.5699 - dense_208_loss: 0.6388 - dense_210_loss: 1.2894 - dense_212_loss: 1.8533 - dense_214_loss: 1.8958 - dense_216_loss: 1.6557 - dense_218_loss: 1.3485 - dense_220_loss: 1.3754 - dense_222_loss: 1.3640 - dense_224_loss: 1.4097 - dense_226_loss: 1.5055 - dense_228_loss: 1.6794 - dense_230_loss: 1.9371 - dense_232_loss: 1.8547 - dense_234_loss: 1.7253 - dense_236_loss: 1.6071 - dense_238_loss: 1.6961 - dense_240_loss: 1.6746 - dense_194_acc: 0.5000 - dense_196_acc: 0.5000 - dense_198_acc: 0.5000 - dense_200_acc: 0.4667 - dense_202_acc: 0.4667 - dense_204_acc: 0.8333 - dense_206_acc: 0.5667 - dense_208_acc: 0.3333 - dense_210_acc: 0.2000 - dense_212_acc: 0.2667 - dense_214_acc: 0.1667 - dense_216_acc: 0.3000 - dense_218_acc: 0.5000 - dense_220_acc: 0

30/30 [==============================] - 1s 37ms/step - loss: 29.9881 - dense_194_loss: 0.4016 - dense_196_loss: 1.1135 - dense_198_loss: 0.8255 - dense_200_loss: 0.6323 - dense_202_loss: 0.5468 - dense_204_loss: 0.2381 - dense_206_loss: 0.5568 - dense_208_loss: 0.6392 - dense_210_loss: 1.2756 - dense_212_loss: 1.8329 - dense_214_loss: 1.9053 - dense_216_loss: 1.6079 - dense_218_loss: 1.3119 - dense_220_loss: 1.3020 - dense_222_loss: 1.3644 - dense_224_loss: 1.3855 - dense_226_loss: 1.4819 - dense_228_loss: 1.6230 - dense_230_loss: 1.4327 - dense_232_loss: 1.8673 - dense_234_loss: 1.7025 - dense_236_loss: 1.5947 - dense_238_loss: 1.7045 - dense_240_loss: 1.6422 - dense_194_acc: 0.5000 - dense_196_acc: 0.5000 - dense_198_acc: 0.5000 - dense_200_acc: 0.4667 - dense_202_acc: 0.4667 - dense_204_acc: 0.8333 - dense_206_acc: 0.5000 - dense_208_acc: 0.4000 - dense_210_acc: 0.1000 - dense_212_acc: 0.2333 - dense_214_acc: 0.2000 - dense_216_acc: 0.3333 - dense_218_acc: 0.4667 - dense_220_acc: 0

30/30 [==============================] - 1s 37ms/step - loss: 24.3305 - dense_194_loss: 0.2736 - dense_196_loss: 1.0729 - dense_198_loss: 0.8130 - dense_200_loss: 0.6405 - dense_202_loss: 0.5150 - dense_204_loss: 0.2109 - dense_206_loss: 0.5549 - dense_208_loss: 0.6215 - dense_210_loss: 1.2226 - dense_212_loss: 1.7651 - dense_214_loss: 1.8561 - dense_216_loss: 0.6269 - dense_218_loss: 0.5372 - dense_220_loss: 1.2207 - dense_222_loss: 1.0495 - dense_224_loss: 1.3483 - dense_226_loss: 1.4674 - dense_228_loss: 1.4573 - dense_230_loss: 0.3596 - dense_232_loss: 1.8286 - dense_234_loss: 1.6610 - dense_236_loss: 1.3229 - dense_238_loss: 1.3093 - dense_240_loss: 0.5955 - dense_194_acc: 0.5000 - dense_196_acc: 0.5000 - dense_198_acc: 0.5000 - dense_200_acc: 0.5000 - dense_202_acc: 0.5000 - dense_204_acc: 0.8333 - dense_206_acc: 0.3667 - dense_208_acc: 0.4667 - dense_210_acc: 0.1333 - dense_212_acc: 0.2667 - dense_214_acc: 0.2333 - dense_216_acc: 0.3333 - dense_218_acc: 0.5000 - dense_220_acc: 0

30/30 [==============================] - 1s 37ms/step - loss: 16.9259 - dense_194_loss: 0.2517 - dense_196_loss: 0.7063 - dense_198_loss: 0.7241 - dense_200_loss: 0.6297 - dense_202_loss: 0.5059 - dense_204_loss: 0.2161 - dense_206_loss: 0.5334 - dense_208_loss: 0.6411 - dense_210_loss: 1.1973 - dense_212_loss: 1.7471 - dense_214_loss: 0.4218 - dense_216_loss: 0.3093 - dense_218_loss: 0.2937 - dense_220_loss: 0.5959 - dense_222_loss: 0.2378 - dense_224_loss: 1.3696 - dense_226_loss: 1.4230 - dense_228_loss: 0.6069 - dense_230_loss: 0.3152 - dense_232_loss: 1.8129 - dense_234_loss: 1.1951 - dense_236_loss: 0.3554 - dense_238_loss: 0.4342 - dense_240_loss: 0.4022 - dense_194_acc: 0.6667 - dense_196_acc: 0.5000 - dense_198_acc: 0.5000 - dense_200_acc: 0.4667 - dense_202_acc: 0.4667 - dense_204_acc: 0.8333 - dense_206_acc: 0.4333 - dense_208_acc: 0.4000 - dense_210_acc: 0.1333 - dense_212_acc: 0.2667 - dense_214_acc: 0.1667 - dense_216_acc: 0.2333 - dense_218_acc: 0.5000 - dense_220_acc: 0

30/30 [==============================] - 1s 37ms/step - loss: 11.3424 - dense_194_loss: 0.2386 - dense_196_loss: 0.4215 - dense_198_loss: 0.6335 - dense_200_loss: 0.6154 - dense_202_loss: 0.4042 - dense_204_loss: 0.2134 - dense_206_loss: 0.4788 - dense_208_loss: 0.5907 - dense_210_loss: 0.3911 - dense_212_loss: 1.7982 - dense_214_loss: 0.3262 - dense_216_loss: 0.2567 - dense_218_loss: 0.2074 - dense_220_loss: 0.2472 - dense_222_loss: 0.2207 - dense_224_loss: 0.3705 - dense_226_loss: 1.3750 - dense_228_loss: 0.2221 - dense_230_loss: 0.2421 - dense_232_loss: 0.8814 - dense_234_loss: 0.4291 - dense_236_loss: 0.2372 - dense_238_loss: 0.2617 - dense_240_loss: 0.2798 - dense_194_acc: 0.6333 - dense_196_acc: 0.5333 - dense_198_acc: 0.5000 - dense_200_acc: 0.4667 - dense_202_acc: 0.5333 - dense_204_acc: 0.8333 - dense_206_acc: 0.4333 - dense_208_acc: 0.4667 - dense_210_acc: 0.1000 - dense_212_acc: 0.2667 - dense_214_acc: 0.2333 - dense_216_acc: 0.2667 - dense_218_acc: 0.4667 - dense_220_acc: 0

In [ ]:
from sklearn.metrics import mean_squared_log_error

In [ ]:
for k in range(0, 21):
    day = LAST_DAY.date() - timedelta(k)
    X = make_aggs(df_[df_.created_at < day], "cell_id", 0, day).values.reshape(1, -1)
    
    y = np.zeros((24, 1, 7))
    group = df_[df_.created_at.dt.date == day].groupby(["final_time", "cell_id"]).created_at.count()
    for i in range(24):
        for j, cell in enumerate(center):
            if i in group:
                if cell in group.loc[i]:
                    y[i][0][j] = group.loc[i, cell]
    print(y)
#     pred = model.predict(np.array([X]))
    

In [427]:
LAST_DAY

Timestamp('2019-02-21 15:59:53.877000')

In [428]:
d

,created_at,pickup_lat,pickup_lng,dropoff_lat,dropoff_lng,user_id,driver_id,ride_distance,canceled_by_client,canceled_by_driver,...,year,is_year_start,is_year_end,dayofyear,dayofweek,week,hour,minute,final_time,is_weekend
ride_id,,,,,,,,,,,,,,,,,,,,,
0000e61e5282484d943291df7266ee33n,2019-01-18 09:18:44.377,49.837013,24.034357,49.832710,23.999216,3300360037004600420041003600300032003500410033...,2913413c2bf748d7932d1d587c8aef13,3.39,0,0,...,2019,0,0,18,4,3,9,18,9,0
000152b506034bcda66a4a813f1408f4n,2018-12-20 02:28:31.017,49.844044,24.024796,49.843155,23.972368,3100360033004100350036003900370036004600410031...,fc111a9c9ac64f7e88af1bff7565e2c1,4.55,0,0,...,2018,0,0,354,3,51,2,28,2,0
0001a52849f141a48f292bbd73b8d7c5n,2018-11-05 22:38:28.870,49.834843,24.022715,49.842907,24.008863,62276F5C7861375C7864305A235C7864665C786230285C...,ec29cf0316764f5d84b93bf28dc3fd58,2.64,0,0,...,2018,0,0,309,0,45,22,38,23,0
000218b0c8494514b78a2730c8e546bdn,2018-12-14 07:53:01.270,49.843437,24.027189,49.840366,24.032738,62277471745C7865335C7864645C78653123296E76405C...,a9a2329425ff413a96ed44fc800cd740,1.36,1,0,...,2018,0,0,348,4,50,7,53,8,0
0002d81308f34226bfd1ed1b59353fd9n,2018-11-02 23:53:50.663,49.840462,24.029787,49.870934,24.027010,62275C786631635C7831345C7862645C7831395C783031...,8fc9bd7d836b49338c303ea149ae8911,4.10,0,0,...,2018,0,0,306,4,44,23,53,0,0
00030efa111e4c50b7d6da117cf4c484n,2018-10-26 21:56:57.693,49.839897,24.037411,49.841515,23.962942,226222222F755C7866375C7862335C7865386F5C783834...,e7b04d20a1c94229b1e24dad4c877162,6.13,0,0,...,2018,0,0,299,4,43,21,56,22,0
000506b3923c4703ab4864e16e080736n,2019-01-21 22:50:36.340,49.837749,24.039181,49.838486,23.972292,3300320041003500410035003800390034004500380043...,a2875773651f4a229bb2aa20410087ba,6.47,0,0,...,2019,0,0,21,0,4,22,50,23,0
000524877c4c4ccba1d16276efb03a22n,2018-09-03 16:19:58.297,49.834614,24.015869,49.866341,24.003979,62272D5C7838625C786531545C725C7862345C7839325C...,7d7916f9007f4053a0ea5b6e0980c506,4.81,0,0,...,2018,0,0,246,0,36,16,19,16,0
000548fdc26e4db3992c7b4f96176511n,2019-01-19 12:17:50.720,49.842392,24.026415,49.841442,24.011053,3200360034003300430034004100380033003300370038...,8ad9ebe3265244a3acdf0770f47a9f8f,1.77,0,0,...,2019,0,0,19,5,3,12,17,12,1


In [426]:
group = df_[df_.created_at.dt.date == day].groupby(["final_time", "cell_id"]).created_at.count()
group

Series([], Name: created_at, dtype: int64)

In [422]:
pred

[array([[-0., -0., -0., -0., -0., -0., -0.]], dtype=float32),
 array([[-0., -0., -0., -0., -0., -0., -0.]], dtype=float32),
 array([[-0., -0., -0., -0., -0., -0., -0.]], dtype=float32),
 array([[-0., -0., -0., -0., -0., -0., -0.]], dtype=float32),
 array([[-0., -0., -0., -0., -0., -0., -0.]], dtype=float32),
 array([[-0., -0., -0., -0., -0., -0., -0.]], dtype=float32),
 array([[-0., -0., -0., -0., -0., -0., -0.]], dtype=float32),
 array([[-0., -0., -0., -0., -0., -0., -0.]], dtype=float32),
 array([[-0., -0., -0., -0., -0., -0., -0.]], dtype=float32),
 array([[-0., -0., -0., -0., -0., -0., -0.]], dtype=float32),
 array([[-0., -0., -0., -0., -0., -0., -0.]], dtype=float32),
 array([[-0., -0., -0., -0., -0., -0., -0.]], dtype=float32),
 array([[-0., -0., -0., -0., -0., -0., -0.]], dtype=float32),
 array([[-0., -0., -0., -0., -0., -0., -0.]], dtype=float32),
 array([[-0., -0., -0., -0., -0., -0., -0.]], dtype=float32),
 array([[-0., -0., -0., -0., -0., -0., -0.]], dtype=float32),
 array([

In [423]:
y

array([[[0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0.]]])

In [416]:
mean_squared_logarithmic_error(np.array([1.]), np.array([3.]))

AttributeError: 'numpy.dtype' object has no attribute 'base_dtype'